# Adrián Gayo Andrés

## Enunciado
Se propone la creación de un prototipo con varios
agentes que permita extraer publicaciones científicas de ArXiv y las publique en X.
En el ITCL se busca desarrollar una herramienta basada en un sistema de agentes cuyo
objetivo principal es permitir que los usuarios accedan a ArXiv a través de una API,
utilizando una interfaz de chat que funcione mediante lenguaje natural. Además, el sistema
debe ofrecer la posibilidad de que los usuarios publiquen, de forma sencilla y en cualquier
momento, los contenidos de una o varias investigaciones en la plataforma X. Este sistema
estará diseñado para ser semiautónomo, pero siempre bajo la supervisión directa del
usuario.


### Imports

In [ ]:
# %pip install arxiv
# %pip install langchain
# %pip install langgraph
# %pip install chromadb 
# %pip install cohere
# %pip install tweepy 
# %pip install gradio
# %pip install openai

In [5]:
import arxiv
import langchain as lc
import langgraph
import tweepy as tw
import openai 
import chromadb as cdb
import cohere as ch
import gradio as gr
import os
from dotenv import load_dotenv
import pandas as pd

print(lc.__version__)


0.3.14


### ArXiV API prueba

In [6]:
# Iinicializamos el cliente de arXiv
arxiv_client = arxiv.Client()

# Query de ejemplo con la palabra "quantum."
search = arxiv.Search(
  query = "quantum",
  max_results = 10,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

# Obtenemos los resultados de la búsqueda
results = arxiv_client.results(search)

# Mostremos los resultados.
for r in arxiv_client.results(search):
  print(r.title)
  

Coming full circle -- A unified framework for Kochen-Specker contextuality
Exact Parent Hamiltonians for All Landau Level States in a Half-flux Lattice
Partonic distribution functions and amplitudes using tensor network methods
Bidirectional microwave-optical conversion using an integrated barium-titanate transducer
Hybridization between surface flat bands and bulk bands in the topological nodal-line semimetal Sn$_{0.15}$NbSe$_{1.75}$ probed via soft-point-contact spectroscopy
Running EFT-hedron with null constraints at loop level
Large moiré superstructure of stacked incommensurate charge density waves
Converse bounds for quantum hypothesis exclusion: A divergence-radius approach
Belavkin-Staszewski Quantum Markov Chains
Sample-based Krylov Quantum Diagonalization


In [11]:
# También podemos descargar el PDF de un artículo.
paper = next(arxiv_client.results(search))
# Creamos un directorio para almacenar los PDFs en caso de que no exista.
os.makedirs("pdfs", exist_ok=True)
# Download the PDF to the PWD with a custom filename.
paper.download_pdf(filename="pdfs/" + paper.title + ".pdf")

'./pdfs/Coming full circle -- A unified framework for Kochen-Specker contextuality.pdf'

### X API prueba (tweepy)

In [19]:
# Nos autenticamos en la API de Twitter
load_dotenv()
twitter_api_key = os.getenv("TWITTER_API_KEY")
twitter_api_secret = os.getenv("TWITTER_API_SECRET")
twitter_access_token = os.getenv("TWITTER_ACCESS_TOKEN")
twitter_access_token_secret = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")

#auth = tw.OAuth1UserHandler(twitter_api_key, twitter_api_secret, twitter_access_token, twitter_access_token_secret)
#api = tw.API(auth, wait_on_rate_limit=True)
x_client = tw.Client(consumer_key=twitter_api_key, consumer_secret=twitter_api_secret, access_token=twitter_access_token, access_token_secret=twitter_access_token_secret)

# Publicamos un tweet de prueba
#api.update_status("Hello, world!")
response = x_client.create_tweet(
    text="Hello, world!"
)
print(f"https://twitter.com/user/status/{response.data['id']}")



Forbidden: 403 Forbidden
Your client app is not configured with the appropriate oauth1 app permissions for this endpoint.

### Inicialización del modelo

In [3]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Parámetros
MAX_TOKENS = 1000
TEMPERATURE = 0.3
MODEL= "gpt-3.5-turbo"
EMBEDDING_MODEL = "gpt-3.5-turbo"

# OpenAI API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Inicializamos el modelo de openai que vamos a utilizar con langchain.
llm = OpenAI(
    model=MODEL,  
    temperature=TEMPERATURE,        
    max_tokens=MAX_TOKENS,        
    openai_api_key=openai_api_key  
)

# Inicializamos también el modelos de embeddings seleccionado.
embeddings = OpenAIEmbeddings(    # (openai embeddings)
    model=EMBEDDING_MODEL,  
    openai_api_key=openai_api_key    
)

#  alternativa para obtener embeddings
#

'sk-1234567890abcdef1234567890abcdef'

In [ ]:
# Se define una función que permita monitorizar el coste de la llamada a la API de OpenAI.
from langchain.callbacks import get_openai_callback

def llamada_monitorizada(llm, texto):
    """ Función que realiza una llamada al modelo de lenguaje especificado y monitoriza el coste de la llamada.
    Args:
        llm (OpenAI): Modelo de lenguaje
        texto (str): Texto de entrada

    Returns:
        response (str): Texto de salida generado por el modelo de lenguaje
    """
    
    with get_openai_callback() as callback:
        response = llm(texto)
        
        # Detalles de uso y coste de la llamada.
        print(f"Tokens usados: {callback.total_tokens}")
        print(f"Coste de la llamada: ${callback.total_cost:.5f}")
        
        return response


### Declaración de herramientas

#### Funciones

In [20]:
def buscar_publicaciones_arxiv(consulta: str, max_resultados: int = 3) -> object:
    """
    Busca publicaciones científicas en arXiv según una consulta del usuario.
    
    Args:
        consulta (str): El término de búsqueda para arXiv.
        max_resultados (int): Número máximo de publicaciones a devolver.

    Returns:
        PDFs de las publicaciones encontradas.
    """
    try:
        # Iinicializamos el cliente de arXiv
        arxiv_client = arxiv.Client()

        # Realizar la consulta a arXiv
        search = arxiv.Search(
            query=consulta,
            max_results=max_resultados,
            sort_by=arxiv.SortCriterion.Relevance
        )
        
        results = arxiv_client.results(search)
        
        if not results:
            return "No se encontraron publicaciones."
        
        for paper in results:
            print(paper.title)
            
        return results
          
    except Exception as e:
        return f"Error al buscar en arXiv: {str(e)}"
    
    
def publicar_en_x(texto: str) -> str:
    """
    Publica un texto en X (Twitter) utilizando la API de Tweepy.
    
    Args:
        texto (str): El texto que se desea publicar en X.
        api_key (str): La API Key de la cuenta de desarrollador de Twitter.
        api_key_secret (str): El API Key Secret asociado.
        access_token (str): El Access Token del usuario.
        access_token_secret (str): El Access Token Secret asociado.
    
    Returns:
        str: Mensaje de éxito o error.
    """
    try:
        load_dotenv()
        twitter_api_key = os.getenv("TWITTER_API_KEY")
        twitter_api_secret = os.getenv("TWITTER_API_SECRET")
        twitter_access_token = os.getenv("TWITTER_ACCESS_TOKEN")
        twitter_access_token_secret = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")

        #auth = tw.OAuth1UserHandler(twitter_api_key, twitter_api_secret, twitter_access_token, twitter_access_token_secret)
        #api = tw.API(auth, wait_on_rate_limit=True)
        x_client = tw.Client(consumer_key=twitter_api_key, consumer_secret=twitter_api_secret, access_token=twitter_access_token, access_token_secret=twitter_access_token_secret)
        
        response = x_client.create_tweet(
            text=texto
        )
        
        return response.status_code
    
    except Exception as e:
        return f"Error al publicar el tweet: {e.response.text}"

#### Tools

In [21]:
from langchain.tools import BaseTool

class ArXiVSearchTool(BaseTool):
    """Herramienta de busqueda de artículos científicos en arXiv."""
    
    name = "ArXiVSearchTool"
    description = (
        "Searches for scientific publications in arXiv related to a specific topic. "
        "It provides a summary of the publications, including title, authors, and links. "
        "Use this when scientific information is needed."
    )

    def _run(self, consulta: str, max_resultados: int = 3) -> str:
        """Synchronous logic for executing the tool."""
        try:
            # Realizar la consulta a arXiv
            return buscar_publicaciones_arxiv(consulta, max_resultados)
        
        except Exception as e:
            return f"An error occurred: {str(e)}"

    async def _arun(self, consulta: str, max_resultados: int = 3) -> str:
        """Asynchronous logic for executing the tool."""
        raise NotImplementedError("This tool does not support asynchronous operations.")


class TwitterPostTool(BaseTool):
    """Herramienta para publicar texto en X(Twitter)."""
    
    name = "TwitterPostTool"
    description = (
        "Posts a message on X(Twitter). "
        "Use this to share a message given a in your X account."
    )
    
    def _run(self, texto: str) -> str:
        """Synchronous logic for executing the tool."""
        try:
            return publicar_en_x(texto)
        
        except Exception as e:
            return f"An error occurred: {str(e)}"
    
    async def _arun(self, texto: str) -> str:
        """Asynchronous logic for executing the tool."""
        raise NotImplementedError("This tool does not support asynchronous operations.")

PydanticUserError: Field 'name' defined on a base class was overridden by a non-annotated attribute. All field definitions, including overrides, require a type annotation.

For further information visit https://errors.pydantic.dev/2.8/u/model-field-overridden

### Grafo Multiagente

In [ ]:
from langgraph.graph import Graph
from langgraph.nodes import Node

graph = Graph()

# Añadimos los nodos de las herramientas al grafo.



### Generación de base de datos vectorial

In [12]:
# Se incializa el cliente y se crea una BBDD (collection)
chroma_client = cdb.Client()

# Crea la colección
collection = chroma_client.create_collection(name="arxiv")

# Añade documentos a la colección
collection.add(
    documents=["ejemplo número 1", "ejemplo número 2", "ejemplo número 3"],
    metadatas=[{"source": "arxiv"}, {"source": "arxiv"}, {"source": "arxiv"}],
    ids=["1", "2", "3"]
)

### Front 

In [16]:
import gradio as gr

def yes_man(message, history):
    if message.endswith("?"):
        return "Yes"
    else:
        return "Ask me anything!"

demo = gr.ChatInterface(
    fn=yes_man,
    title="MULTIAGENTE ARXIV RAG",
    description="Say hello to someone!",
)

demo.launch()


c:\Users\adria\anaconda3\Lib\site-packages\gradio\components\chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
